#### Code Agents with `smolagents 🤗`

In [ ]:
# Default Setup Cell.
# It imports environment variables, define 'devtools.debug" as a buildin, set PYTHONPATH and autorelaod
# Copy it in other Notebooks

import builtins

from devtools import debug
from dotenv import load_dotenv

builtins.debug = debug
load_dotenv(verbose=True)

%load_ext autoreload
%autoreload 2
%reset -f

!export PYTHONPATH=":./python"

In [ ]:
from typing import Optional

from IPython.display import Markdown, display
from smolagents import (
    CodeAgent,
    DuckDuckGoSearchTool,
    LiteLLMModel,
    ToolCallingAgent,
    VisitWebpageTool,
    tool,
)

from src.ai_core.llm import LlmFactory

MODEL_ID = "gpt_4omini_openai"
llm_factory = LlmFactory(llm_id=MODEL_ID, llm_params={"temperature": 0.7})
llm = LiteLLMModel(model_id=llm_factory.get_litellm_model_name(), **llm_factory.llm_params)

### Simple  `smolagents 🤗` CodeAgent

...  but quite impressive !

In [ ]:
agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=llm)
agent.run("How many seconds would it take for a leopard at full speed to run through Pont des Arts?")

### Simple ToolCalling Agent

In [ ]:
@tool
def get_weather(location: str, celsius: Optional[bool] = False) -> str:
    """Get weather in the next days at given location.
    Secretly this tool does not care about the location, it hates the weather everywhere.

    Args:
        location: the location
        celsius: the temperature
    """
    return "The weather is UNGODLY with torrential rains and temperatures below -10°C"


agent = ToolCallingAgent(tools=[get_weather], model=llm)
print(agent.run("What's the weather like in Paris?"))

### RAG System with `smolagents 🤗` and `langchain 🦜️🔗`


In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

from src.ai_core.embeddings import EmbeddingsFactory
from src.ai_core.vector_store import VectorStoreFactory
from src..ai_extra.smolagents_chain import RetrieverTool

# Usual RAG stuff : load document, split it, and add chunks into a vectorstore

documents = TextLoader("use_case_data/other/state_of_the_union.txt").load()
texts = RecursiveCharacterTextSplitter(chunk_size=2000).split_documents(documents)
vs = VectorStoreFactory(id="InMemory", embeddings_factory=EmbeddingsFactory()).vector_store
_ = vs.add_documents(texts)
vs_retriever = vs.as_retriever(k=10)


# Initialize agent
def create_rag_agent(retriever):
    retriever_tool = RetrieverTool(retriever)
    return CodeAgent(tools=[retriever_tool], model=llm, max_steps=4)


agent = create_rag_agent(vs_retriever)
response = agent.run("What did the president say about Ketanji Brown Jackson")
debug(response)

### Understand how it works.... 

In [ ]:
display(Markdown(agent.system_prompt_template))

#### Langchain Tool with `smolagents 🤗` Search Tool

In [ ]:
# search_tool = Tool.from_langchain(load_tools(["serpapi"])[0])

# agent = CodeAgent(tools=[search_tool], model=HfApiModel())

# agent.run("How many more blocks (also denoted as layers) are in BERT base encoder compared to the encoder from the architecture proposed in Attention is All You Need?")

In [ ]:
agent = CodeAgent(
    tools=[VisitWebpageTool()],
    model=llm,
    additional_authorized_imports=["requests", "markdownify", "bs4"],
    # use_e2b_executor=True,
)

agent.run("Qui est l'actuel premier ministre en France ?")

### Self Correcting Text to SQL with `smolagents 🤗`

In [ ]:
from sqlalchemy import (
    Column,
    Float,
    Integer,
    MetaData,
    String,
    Table,
    create_engine,
    insert,
    inspect,
    text,
)

engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

table_name = "receipts"
receipts = Table(
    table_name,
    metadata_obj,
    Column("receipt_id", Integer, primary_key=True),
    Column("customer_name", String(16), primary_key=True),
    Column("price", Float),
    Column("tip", Float),
)
metadata_obj.create_all(engine)

rows = [
    {"receipt_id": 1, "customer_name": "Alan Payne", "price": 12.06, "tip": 1.20},
    {"receipt_id": 2, "customer_name": "Alex Mason", "price": 23.86, "tip": 0.24},
    {"receipt_id": 3, "customer_name": "Woodrow Wilson", "price": 53.43, "tip": 5.43},
    {"receipt_id": 4, "customer_name": "Margaret James", "price": 21.11, "tip": 1.00},
    {"receipt_id": 5, "customer_name": "John Doe", "price": 100.00, "tip": 10.00},
]
for row in rows:
    stmt = insert(receipts).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

table_name = "waiters"
receipts = Table(
    table_name,
    metadata_obj,
    Column("receipt_id", Integer, primary_key=True),
    Column("waiter_name", String(16), primary_key=True),
)
metadata_obj.create_all(engine)

rows = [
    {"receipt_id": 1, "waiter_name": "Corey Johnson"},
    {"receipt_id": 2, "waiter_name": "Michael Watts"},
    {"receipt_id": 3, "waiter_name": "Michael Watts"},
    {"receipt_id": 4, "waiter_name": "Margaret James"},
]
for row in rows:
    stmt = insert(receipts).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

In [ ]:
updated_description = """Allows you to perform SQL queries on the table. Beware that this tool's output is a string representation of the execution output.
It can use the following tables:"""

inspector = inspect(engine)
for table in ["receipts", "waiters"]:
    columns_info = [(col["name"], col["type"]) for col in inspector.get_columns(table)]

    table_description = f"Table '{table}':\n"

    table_description += "Columns:\n" + "\n".join([f"  - {name}: {col_type}" for name, col_type in columns_info])
    updated_description += "\n\n" + table_description

print(updated_description)

In [ ]:
from smolagents import tool


@tool
def sql_engine(query: str) -> str:
    """Allows you to perform SQL queries.

    Args:
        query: The query to perform. This should be correct SQL.
    """
    output = ""
    with engine.connect() as con:
        rows = con.execute(text(query))
        for row in rows:
            output += "\n" + str(row)
    return output


sql_engine.description = updated_description

In [ ]:
agent = CodeAgent(tools=[sql_engine], model=llm)
agent.run("Can you give me the name of the client who got the most expensive receipt?")